In [1]:
%matplotlib qt

import numpy as np
import hyperspy.api as hs
import matplotlib.pyplot as plt
import cv2

from matplotlib import colors

In [2]:
# Add '.tiff' file directory here
# data_file_dir = "/media/ivan/Ivan/jad_spectral_2/src_Specim-FX17e-076900055547_02.tiff"

# 
data_file_dir = "/media/ivan/Ivan/jad_spectral_3/src_Specim-FX17e-076900055547_43.tiff"

#data_file_dir = "/media/ivan/Ivan/jad_spectral/src_Specim-FX17e-076900055547_00.tiff"
spectral_data = []

_, spectral_data = cv2.imreadmulti(
    filename=data_file_dir,
    mats=spectral_data,
    flags=cv2.IMREAD_UNCHANGED
)

spectral_data = np.asarray(spectral_data)
spectral_data = np.transpose(spectral_data, (0, 2, 1))

In [3]:

# region_of_interest = spectral_data[980:1000, 370:415, :]
# print(region_of_interest)
# empty = spectral_data[980:1000, 390:395, 57:61]
# print(empty)

In [4]:
# red = spectral_data[980:1000, 409:415, 57:61]
# print(red)

In [5]:
label_correspondences = {
    "meat": 0,
    "fat": 1,
    "conveyor": 2,
    "plastic": 3
}

# Add label correspondences, change integer values to change heat map
color_correspondences = {
    label_correspondences["meat"]: "g",
    label_correspondences["fat"]: "w",
    label_correspondences["conveyor"]: "b",
    label_correspondences["plastic"]: "r",
}

In [6]:
# Initialize segmented image
segmented_image = np.zeros(spectral_data.shape[:2], dtype=np.uint8)

# Conveyor classification
conveyor_pixels = np.where((spectral_data[:, :, 30] - spectral_data[:, :, 25]) > 0.0)
segmented_image[conveyor_pixels[0], conveyor_pixels[1]] = 2

In [7]:
# Fat classification
# fat_condition = ((spectral_data[:, :, 80] - spectral_data[:, :, 61])/(80 - 61) - (spectral_data[:, :, 61] - spectral_data[:, :, 53])/(61 - 53))
# fat_pixels = np.where(
#     np.logical_and(
#         np.logical_or(fat_condition < -0.005, fat_condition > 0.005),
#         (spectral_data[:, :, 30] - spectral_data[:, :, 25]) < 0.0
#     )
# )

# Fat classification
fat_condition = ((spectral_data[:, :, 80] - spectral_data[:, :, 61])/(80 - 61) - (spectral_data[:, :, 61] - spectral_data[:, :, 53])/(61 - 53))
extra_condition = ((spectral_data[:, :, 154] - spectral_data[:, :, 112])/(154 - 112))
fat_pixels = np.where(
    np.logical_and(
        np.logical_and(
            fat_condition > -0.0025,
            (spectral_data[:, :, 30] - spectral_data[:, :, 25]) < 0.0
        ),
        (spectral_data[:, :, 61] - spectral_data[:, :, 57]) > 0.0015
    )
)
# fat_pixels = np.where(
#     np.logical_and(
#         np.logical_and(
#             np.logical_and(
#                 fat_condition > -0.0025,
#                 (spectral_data[:, :, 30] - spectral_data[:, :, 25]) < 0.0
#             ),
#             (spectral_data[:, :, 61] - spectral_data[:, :, 57]) > 0.0015
#         ),
#         extra_condition > 0.00025
#     )
# )

segmented_image[fat_pixels[0], fat_pixels[1]] = 1

In [8]:
# # Plastic classification
# second_condition = (spectral_data[:, :, 61] + spectral_data[:, :, 57]) > 0.05
# first_condition = (spectral_data[:, :, 61] - spectral_data[:, :, 57]) < 0.00045 # 0.00045

# #slope1 = (spectral_data[:, :, 61]-spectral_data[:, :, 57])/4 
# #slope2 = (spectral_data[:, :, 80]-spectral_data[:, :, 61])/19
# #third_condition = (np.abs(slope2) - np.abs(slope1)) > 1

# plastic_pixels = np.where(np.logical_and(first_condition, second_condition))
# segmented_image[plastic_pixels[0], plastic_pixels[1]] = 3

# Plastic classification
light_check = (spectral_data[:, :, 61] + spectral_data[:, :, 57]) > 0.05
red_check = (spectral_data[:, :, 61] - spectral_data[:, :, 57]) < 0.00045
crease_check = (spectral_data[:, :, 153] - spectral_data[:, :, 150]) < 0.0
extra_check = (spectral_data[:, :, 60] - spectral_data[:, :, 53]) > 0.001
extra_check = False
plastic_pixels = np.where(
    np.logical_and(
        np.logical_and(red_check, light_check),
        np.logical_or(crease_check, extra_check)
    )    
)
segmented_image[plastic_pixels[0], plastic_pixels[1]] = 3

In [9]:
print(3 in segmented_image)

False


In [10]:
# Visualization
color_map = colors.ListedColormap([
    color_correspondences[0],
    color_correspondences[1],
    color_correspondences[2],
    color_correspondences[3]
])
print([
    color_correspondences[0],
    color_correspondences[1],
    color_correspondences[2],
    color_correspondences[3]
])

plt.imshow(segmented_image, interpolation="nearest", cmap=color_map, vmin=0, vmax=4)
plt.show()

['g', 'w', 'b', 'r']
